# SniffNets

This notebook contains the implementation of the SniffNets models created for clasifying the codification of signals deteced from artificial noses. The experiments in this notebook were used in the article [Deep learning models for classification of gases detected by sensor arrays of artificial nose](https://sol.sbc.org.br/index.php/eniac/article/view/9339)

* Disclaimer: Unfortunately the Coffee dataset is not publicly available.


In [ ]:
# Run this block it you are using google colaboratory and desire to save the results in
# google drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Run this block it you are using google colaboratory
!git clone https://github.com/IsmaelCesar/SniffNets.git

In [1]:
from sklearn.utils import shuffle
from data_loading import load_and_split, split_datasamples_by_sensors, standardize_data
from data_loading import dataset_classes_number, dataset_wine_classes_number
from experiment_procedures import resettv, get_dataset_names_and_indexes, get_wine_dataset_names_and_indexes
from models import sniffnet, sniffresnet, sniffmultinose
from evaluation import evaluate_model, save_results_into_filesystem
# from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.utils import to_categorical

## Running models

In [2]:
batch_size  = 10
epochs = 20
lr = .001

ds_names_and_indexes = get_dataset_names_and_indexes()
print("Dict: ",ds_names_and_indexes)
print("Names: ",  list(ds_names_and_indexes.keys()))

Dict:  {'fonollosa': [0, 1, 2, 3, 4], 'turbulent_gas_mixtures': [0], 'windtunnel': [0, 1, 2, 3, 4, 5]}
Names:  ['fonollosa', 'turbulent_gas_mixtures', 'windtunnel']


In [3]:
def run_experiments(batch_size, epochs, ds_names_and_indexes, select_model=0, read_wine=False):
    ds_names = list(ds_names_and_indexes.keys())
    
    for ds_name in ds_names:
        indexes = ds_names_and_indexes[ds_name]
        
        n_classes = 0 
        
        if read_wine: 
            n_classes = dataset_wine_classes_number(ds_name)
        else:
            n_classes = dataset_classes_number(ds_name)
        
        for ds_idx in indexes:
            train_data, train_labels, test_data,test_labels = load_and_split(ds_name, ds_idx=ds_idx, 
                                                                             read_wine_datasets=read_wine)

            train_data, test_data = standardize_data(train_data,test_data)
            
            if select_model == 0:
                model = sniffnet(input_shape,n_classes)
            elif select_model == 1:
                model = sniffresnet(input_shape,n_classes)
            elif select_model == 2: 
                train_data = split_datasamples_by_sensors(train_data)
                test_data = split_datasamples_by_sensors(test_data)
                
                model = sniffmultinose(input_shape, n_classes)
                
            
            model.compile(loss="categorical_crossentropy",
                          optimizer=keras.optimizers.SGD(lr=lr,momentum=.9),
                          metrics=['accuracy'])
            
            train_labels  = to_categorical(train_labels,n_classes)
            test_labels = to_categorical(test_labels, n_classes)
            
            print("[INFO] Training Network")
            H = model.fit(train_data,train_labels,batch_size=batch_size,
                          epochs=epochs,verbose=1,validation_data=(test_data,test_labels))

            evaluate_model(test_data,test_labels,batch_size,model, n_epochs, H,
                           experiments_folder, dataset_name, sub_dataset_name, 
                           model_folder, save_resulst=False)            

## Run experiments on Fonollosa, Windtunnel and Turbulent Gas Mixtures Dataset

### SniffConvNet

In [ ]:
run_experiments(batch_size, epochs, ds_names_and_indexes, select_model=0)

### SniffResnet

In [ ]:
run_experiments(batch_size, epochs, ds_names_and_indexes, select_model=1)

### SniffMultinose

In [ ]:
run_experiments(batch_size, epochs, ds_names_and_indexes, select_model=2)

## Run experiments on the Wine datasets

In [ ]:
ds_wine_names_and_indexes = get_wine_dataset_names_and_indexes()

### SniffConvNet

In [ ]:
run_experiments(batch_size, epochs, ds_wine_names_and_indexes, select_model=0, read_wine=True)

### SniffResnet

In [ ]:
run_experiments(batch_size, epochs, ds_wine_names_and_indexes, select_model=1, read_wine=True)

### SniffMultinose

In [ ]:
run_experiments(batch_size, epochs, ds_wine_names_and_indexes, select_model=1, read_wine=True)